# Training a CNN Model for ASL Detection

This notebook demonstrates how to build and train a Convolutional Neural Network (CNN) to classify static ASL signs. The trained model will be used for real-time detection of hand gestures.

## Import Required Libraries

This code cell imports all the necessary libraries for building and training the CNN model, including TensorFlow/Keras components.

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

## Define Data Generators Function

This function creates training and validation data generators that will automatically load, preprocess, and batch the images for training.

In [9]:
def prepare_data_generators(data_dir, target_size=(224, 224), batch_size=32):
    """
    Prepare training and validation data generators.
    
    Args:
        data_dir (str): Path to the dataset directory
        target_size (tuple): Target image size for resizing
        batch_size (int): Batch size for training
        
    Returns:
        tuple: (train_generator, val_generator)
    """
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

## Define CNN Architecture

This function creates the Convolutional Neural Network architecture with multiple convolutional layers, pooling layers, and dropout for regularization.

In [10]:
def create_cnn_model(input_shape=(224, 224, 3), num_classes=26):
    """
    Create a CNN model for ASL classification.
    
    Args:
        input_shape (tuple): Shape of input images
        num_classes (int): Number of classes (26 for A-Z)
        
    Returns:
        tensorflow.keras.Model: Compiled CNN model
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## Load Preprocessed Data

This function loads the preprocessed dataset and prepares the data generators for training and validation.

In [11]:
def load_preprocessed_data(data_dir):
    """
    Load preprocessed dataset for training.
    
    Args:
        data_dir (str): Path to the dataset directory
        
    Returns:
        tuple: (train_generator, val_generator) - Data generators for training and validation
    """
    train_generator, val_generator = prepare_data_generators(data_dir)
    print("Data generators ready for training.")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")
    print(f"Number of classes: {train_generator.num_classes}")
    return train_generator, val_generator

## Train the Model

This function trains the CNN model using the prepared data generators and returns the training history for analysis.

In [12]:
def train_model(model, train_generator, val_generator, epochs=10):
    """
    Train the CNN model.
    
    Args:
        model: The CNN model to train
        train_generator: Training data generator
        val_generator: Validation data generator
        epochs (int): Number of training epochs
        
    Returns:
        History: Training history object containing loss and accuracy metrics
    """
    print(f"Starting training for {epochs} epochs...")
    
    # Add EarlyStopping callback
    early_stop = EarlyStopping(
        monitor='val_loss',      # You can also use 'val_accuracy'
        patience=2,             # Stop after 2 epochs with no improvement
        restore_best_weights=True
    )

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[early_stop],
        verbose=1
    )
    
    print("Training completed!")
    return history

## Save the Trained Model

This function saves the trained model to the specified path for later use in inference and real-time detection.

In [13]:
def save_model(model, model_path):
    """
    Save the trained model to a specified path.
    
    Args:
        model: The trained CNN model
        model_path (str): Path where the model should be saved
    """
    model.save(model_path)
    print(f"Model saved successfully to: {model_path}")
    
    # Also save model summary
    model_summary_path = model_path.replace('.h5', '_summary.txt')
    with open(model_summary_path, 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
    print(f"Model summary saved to: {model_summary_path}")

## Complete Training Pipeline Example

This cell demonstrates how to use all the functions together to train a complete ASL detection model.

In [ ]:

# Configuration
DATA_DIR = "../data/asl_alphabet_train/asl_alphabet_train"  # Path to your dataset
MODEL_SAVE_PATH = "../models/asl_cnn_model.h5"
EPOCHS = 20

# Step 1: Load the data
data_generators = prepare_data_generators(DATA_DIR)
train_generator, val_generator = data_generators

# Step 2: Create the model
model = create_cnn_model(input_shape=(224, 224, 3), num_classes=train_generator.num_classes)

# Step 3: Display model architecture
model.summary()

# Step 4: Train the model
history = train_model(model, train_generator, val_generator, epochs=EPOCHS)

# Step 5: Save the trained model
# Ensure the models directory exists before saving
os.makedirs("../models", exist_ok=True)
MODEL_SAVE_PATH = "../models/asl_cnn_model.h5"
model.save(MODEL_SAVE_PATH)
print(f"Model saved successfully to: {MODEL_SAVE_PATH}")

print("Training pipeline completed successfully!")

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,172,701 (42.62 MB)

 Trainable params: 11,172,701 (42.62 MB)

 Non-trainable params: 0 (0.00 B)

Starting training for 20 epochs...
Epoch 1/20
Epoch 1/20
   9/2175 ━━━━━━━━━━━━━━━━━━━━ 29:40 822ms/step - accuracy: 0.0280 - loss: 4.0907

KeyboardInterrupt: 